In [428]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import logging
import datetime
import pickle

logger = logging.getLogger(__name__)

data = pickle.load(open("./data.pkl",'rb'))
STATES = ["S", "I", "R"]

## Getting Simulator Data

In [429]:
class get_data():
    
    def __init__(self, data):
        self.c_data = data
        self.data = self.clean_data()
        
    def clean_data(self):
        for d in range(len(data)-1, -1, -1):
            if data[d]['event_type'] != 'encounter':
                data.pop(d)
        data_clean = data
        return data_clean
   
    def time_in_range(self, start, end, xs,xe):
        #Return true if x is in the range [start, end]
        if start <= end:
            return start<= xs and xe <= end
        else:
            
            return start<= xs or xe <= end

    def max_time(self): 

        # Initialize maximum time 
        #print(self.data[0])
        max_time1 = self.data[0]['time']
        for d in self.data:
            if d['time'] > max_time1: 
                #print(type(d['time']))
                max_time1 = d['time']
        return max_time1
        
    def time_slice(self):
        # The Time Span is from days chosen by user to current date time
        # The date-time of encounter + duration should fall in the time slice
        #self.clean_data()
        time_slice_start = self.max_time() - datetime.timedelta(days = int(input("Enter Number of Days of Simulation")))
        #print(time_slice_start)
        time_slice_end = self.max_time()
        #print(time_slice_end)
        timestep = int(input("Enter timestep"))
        
        dfinal=[]
        interactions={}
        cdata = self.data
        while(time_slice_start < time_slice_end):
            dlist=[]
            for d in cdata:
                interactions={}
                time_interact_start = d['time']
                if (d['time'] >= time_slice_start) and (d['time'] < (time_slice_start+datetime.timedelta(hours=timestep))):
                    interactions['infectiousness']=d['payload']['unobserved']['human1'].get('infectiousness')
                    interactions['human_id']=d['human_id']
                    interactions['other_human_id']=d['payload']['unobserved']['human2_id']
                    interactions['human1_isinfected']=d['payload']['unobserved']['human1']['is_infected']
                    interactions['human2_isinfected']=d['payload']['unobserved']['human2']['is_infected']
                    dlist.append(interactions)
            dfinal.append(dlist)  
            time_slice_start+=datetime.timedelta(hours = int(timestep))
        print(len(dfinal))
        print(dfinal)        
        return dfinal

In [430]:
gd = get_data(data)
sir_data = gd.time_slice()

Enter Number of Days of Simulation 1
Enter timestep 1


24
[[{'infectiousness': 0, 'human_id': 8, 'other_human_id': 50, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 8, 'other_human_id': 24, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 8, 'other_human_id': 33, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 8, 'other_human_id': 96, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 8, 'other_human_id': 20, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 8, 'other_human_id': 11, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 8, 'other_human_id': 62, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 8, 'other_human_id': 73, 'human1_isinfected': True, 'human2_isinfected': True}, {'infectiousness': 0, 'human_id': 11, 'other_human_id': 11, 'human1_isinfected': Tr

## Model

In [531]:
def get_dummies(states):
    probas = np.zeros(states.shape + (3,))
    for s in [0,1,2]:
        probas[:,:,s] = (states==s)*1
    assert np.all(probas.argmax(axis=2) == states)
    return probas


def get_infection_probas(states, transmissions):
    """
    - states[i] = state of i
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = 1 - prod_{j: state==I} [1 - lambda_ij]
    """
    print("states:",states)
    infected = (states == 1)
    print("infected:",infected)
    N = len(states)
    infection_probas = np.zeros(N)
    for i in range(N):
#         rates = np.array([
#             rate for i0, j, rate in transmissions
#             if i0 == i and infected[j]
#         ])
        rates = np.array([])
        print(transmissions)
        for i0, j, rate in transmissions: 
                if i0 == i and infected[j]:
                     rates.append(rate) 
            
        infection_probas[i] = 1 - np.prod(1 - rates)
    return infection_probas


def propagate(current_states, infection_probas, recover_probas):
    """
    - current_states[i] = state of i
    - infection_probas[i]  = proba that i get infected (if susceptible)
    - recover_probas[i] = proba that i recovers (if infected)
    """
    next_states = np.zeros_like(current_states)
    for i, state in enumerate(current_states):
        if (state == 0):
            infected = np.random.rand() < infection_probas[i]
            next_states[i] = 1 if infected else 0
        elif (state == 1):
            recovered = np.random.rand() < recover_probas[i]
            next_states[i] = 2 if recovered else 1
        else:
            next_states[i] = 2
    return next_states


class EpidemicModel():
    def __init__(self, initial_states):
        self.N = len(initial_states)
        self.initial_states = initial_states

    def time_evolution(self, recover_probas, transmissions, print_every=10):
        """Run the simulation where
        - recover_probas[i] = mu_i time-independent
        - transmissions[t] = list of t, i, j, lambda_ij(t)
        - states[t, i] = state of i at time t
        """
        # initialize states
        T = len(transmissions)
        states = np.empty((T + 1, self.N))
        states[:] = np.nan
        states[0] = self.initial_states
        # iterate over time steps
        for t in range(T):
            if (t % print_every == 0):
                print(f"t = {t} / {T}")
            infection_probas = get_infection_probas(states[t], transmissions[t])
            states[t+1] = propagate(states[t], infection_probas, recover_probas)
        self.states = states
        self.probas = get_dummies(states)    
    
class run_model(EpidemicModel):
    def __init__(self, data, mu, initial_states = None):
        self.data = data
        self.mu = mu
        self.recover_probas = self.recover_probs()
        self.transmissions = transmissions
        
        if initial_states is None:
            initial_states = []
            for i in range(len(self.data)):
                if len(self.data[i]) != 0:
                    for j in range(len(self.data[i])):
                        initial_states.append(self.data[i][j]['human1_isinfected'])
                else:
                    initial_states.append(0)
        super().__init__(initial_states)

        
    def generate_transmissions(self):
        transmissions = []
        for i in range(len(self.data)):
            if len(self.data[i]) != 0:
                for j in range(len(self.data[i])):
                    transmissions.append(self.data[i][j]['infectiousness'])
            else:
                transmissions.append(0)
        
    def recover_probs(self):
        recover_probas = self.mu*np.ones(len(self.data))

    def run(self, print_every=10):
        print("Generating transmissions")
        self.generate_transmissions()
        print("Running simulation")
        self.time_evolution(
            self.recover_probas, self.transmissions, print_every=print_every
        )

In [533]:
model = run_model(sir_data, 0.01)
model.run()

Generating transmissions
Running simulation
t = 0 / 99
states: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.]
infected: [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False False False False
 False False False False False False False False False False False False
 False False False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True]
0


TypeError: 'int' object is not iterable

## Inference

In [ ]:
def get_infection_probas(probas, transmissions):
    """
    - probas[i,s] = P_s^i(t)
    - transmissions = array/list of i, j, lambda_ij
    - infection_probas[i]  = sum_j lambda_ij P_I^j(t)
    """
    N = probas.shape[0]
    infection_probas = np.zeros(N)
    for i in range(N):
        rates = np.array([
            probas[j, 1]*rate
            for i0, j, rate in transmissions if i0 == i
        ])
        infection_probas[i] = rates.sum()
    return infection_probas


def propagate(probas, infection_probas, recover_probas):
    """
    - probas[i,s] = P_s^i(t)
    - infection_probas[i]  = proba that i get infected (if susceptible)
    - recover_probas[i] = proba that i recovers (if infected)
    """
    next_probas = np.zeros_like(probas)
    next_probas[:, 0] = probas[:, 0]*(1 - infection_probas)
    next_probas[:, 1] = probas[:, 1]*(1 - recover_probas) + probas[:, 0]*infection_probas
    next_probas[:, 2] = probas[:, 2] + probas[:, 1]*recover_probas
    return next_probas


class InferenceModel():
    def __init__(self, initial_probas):
        self.N = len(initial_probas)
        self.initial_probas = initial_probas
       

    def time_evolution(self, recover_probas, transmissions, print_every=10):
        """Run the simulation where
        - recover_probas[i] = mu_i time-independent
        - transmissions[t] = list of t, i, j, lambda_ij(t)
        - probas[t, i, s] = state of i at time t
        """
        # initialize states
        T = len(transmissions)
        probas = np.zeros((T + 1, self.N, 3))
        probas[0] = self.initial_probas
        # iterate over time steps
        for t in range(T):
            if (t % print_every == 0):
                print(f"t = {t} / {T}")
            infection_probas = get_infection_probas(probas[t], transmissions[t])
            probas[t+1] = propagate(probas[t], infection_probas, recover_probas)
        self.probas = probas
        self.states = probas.argmax(axis=2)

#     def plot_states(self, t):
#         fig, ax = plt.subplots(1, 1, figsize=(5, 5))
#         for idx, state in enumerate(STATES):
#             ind = np.where(self.states[t] == idx)
#             ax.scatter(self.x_pos[ind], self.y_pos[ind], label=state)
#         ax.set(title="t = %d" % t)
#         ax.legend()

#     def plot_probas(self, t):
#         fig, axs = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
#         for s, (ax, state) in enumerate(zip(axs, STATES)):
#             ax.scatter(self.x_pos, self.y_pos, c=self.probas[t, :, s],
#                        cmap="Blues", vmin=0, vmax=1)
#             ax.set(title=state)
#         fig.tight_layout()

#     def get_counts(self):
#         counts = self.probas.sum(axis=1)
#         return pd.DataFrame(counts, columns=STATES)



In [ ]:
def generate_initial_probas(M, states):
    freqs = [np.mean(states==s) for s in [0,1,2]]
    print("freqs = ", freqs)
    N = len(states)
    initial_probas = np.broadcast_to(freqs, (N, 3)).copy()
    observations = np.random.choice(N, M, replace=False)
    for i in observations:
        s = int(states[i])
        initial_probas[i] = np.zeros(3)
        initial_probas[i, s] = 1.
    return initial_probas

# we observe M=20 persons at time t_obs=50
t_obs = 50
initial_probas = generate_initial_probas(M=20, states=model.states[t_obs])
infer = InferenceModel(initial_probas)

In [ ]:
infer.time_evolution(model.recover_probas, model.transmissions[t_obs:], print_every=100)